## Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import copy
import numpy as np
import pandas as pd
import pulp as pulp
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import grid_search
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score,recall_score
from random import randrange

## Helper Functions

In [ ]:
def strToBin(lst):
    """
        Convert a list of str into binary
        Input: e.g. ['1', '0']
        Output: e.g. [1, 0]
    """
    for i in range(len(lst)):
        lst[i]=eval(lst[i])
    return lst

def CalcIV(x,y):
    """
    Calculate the information gain between two columns of binary data
    
    """
    N_0  = np.sum(y==0)
    if N_0==0:
        N_0=1      
    N_1 = np.sum(y==1)
    if N_1==0:
        N_1=1     
    N_0_group = np.zeros(2)
    N_1_group = np.zeros(2)
    
    # +1 to avoid dividing by 0
    for i in range(2):
        N_0_group[i] = len(y[(x == i) & (y == 0)])
        if N_0_group[i]==0:
            N_0_group[i]=1
        N_1_group[i] = len(y[(x == i) & (y == 1)])
        if N_1_group[i]==0:
            N_1_group[i]=1  
    iv = np.sum((N_0_group/N_0 - N_1_group/N_1) * np.log((N_0_group/N_0)/(N_1_group/N_1)))  
    return iv

def spe_score(y_true,y_pred):
    """
    Calculate the specificity score
    """
    N_neg = sum(y_true==0)
    TN = sum((y_true|y_pred)==0)
    return 1.*TN/N_neg

def getMean(data):
    meanValue = np.round(np.mean(data),3)
    return meanValue

## Data Initialize

In [ ]:
def dataInitialize(file):
    """
    Initialize the data file
    Input:
        file: name of the data file, i.e. Worm/CE-BP.txt
    Outputs:
        x: data
        y: labels
        features: names of features
    """
    data_file = 'Datasets/GeneratedDatasets/ExperimentalDatasets/' + file
    with open(data_file,'r') as f:
        lines = f.readlines()
    y = np.array(strToBin(lines[-1].split(',')[1:-1]))
    x = np.zeros((len(y),len(lines)-2))
    features = []
    for i,line in enumerate(lines[1:-1]):
        features.append(line.split(',')[0])
        x[:,i] =  np.array(strToBin(line.split(',')[1:-1])) 
    return x, y, features

def pathInitialize(file,features):
    """
    Initialize the path file
    Inputs:
        file: name of the path file, i.e. Worm/CE-Path-BP.txt
    Outputs:
        allPath: list of distinct longest paths
        parDict: dictionary of {feature: [parents]}, used in SHSEL method
    """
    path_file = 'Datasets/GeneratedDatasets/GOPath/' + file.split('-')[0] + '-Path-' + file.split('-')[1]
    allPath = []
    with open(path_file,'r') as f:
        line = f.readline()
        while line:
            path = line.split(',')[:-1]
            allPath.append(path)
            line = f.readline()
    parDict = dict()
    for f in features:
        parDict[f] = set()
    for path in allPath:
        for i,node in enumerate(path):
            if i > 0:
                parDict[node].add(path[i-1])
    for l in parDict:
        parDict[l] = list(parDict[l])
    return allPath, parDict

def randomSplit(x,y,features, frac=0.8):
    """
    Random Train/test data split, then calculate IG for each feature in training set
    Inputs:
        x: original data
        y: original labels
        features: names of features
        frac: train-test split ratio
    Outputs:
        x_train: training data
        x_test: testing data
        y_train: training labels
        y_test: testing labels
        IVdict: dictionary of {feature:IG}
    """
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1-frac)
    IVdict = dict()
    for i,feature in enumerate(features):
        IVdict[feature] = CalcIV(x_train[:,i], y_train)
    return x_train, x_test, y_train, y_test, IVdict

## Modeling Step

In [ ]:
def train_test_model(clf, X_train, y_train, X_test, y_test):
    """
    Calculate the model metrics GMean
    Inputs:
        clf: type of classifier
        X_train: training data (numpy matrix)
        X_test: test data (numpy array)
        y_train: training labels (numpy matrix)
        y_test: test labels (numpy array)
    """
    # Fit a model by providing X and y from training set
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    sen = np.round(recall_score(y_test, y_test_pred),3)
    spe = np.round(spe_score(y_test, y_test_pred),3)
    gm = np.round(1.*np.sqrt(sen*spe),3)
    return gm

def modeling(x_train, x_test, y_train, y_test):
    """
    Calculate the metrics GMean(sqrt(sen*spe)) of Naive Bayes classifier plus 5-fold cross validation
    Inputs:
        x_train: training data (numpy matrix)
        x_test: test data (numpy array)
        y_train: training labels (numpy matrix)
        y_test: test labels (numpy array)
    """
    bnb = BernoulliNB()
    clf = grid_search.GridSearchCV(bnb, [{'alpha':[1]}], cv=10, scoring='roc_auc')
    gm_nb = train_test_model(clf, x_train, y_train, x_test, y_test)
    return gm_nb

## No Feature Selection

In [ ]:
def NOFS(x_train, x_test, y_train, y_test):
    """
    Complete feature set, without feature selection
    """
    gm = modeling(x_train,x_test,y_train,y_test)
    return gm

## ReliefF

In [ ]:
def distanceNorm(Norm,D_value):
    # Norm for distance
    if Norm == 1:
        counter = np.absolute(D_value)
        counter = np.sum(counter)
    elif Norm == 2:
        counter = np.power(D_value,2)
        counter = np.sum(counter)
        counter = np.sqrt(counter)
    else:
        counter = np.absolute(D_value)
        counter = np.max(counter)
    return counter

def fit(features,labels,iter_ratio,k,norm):
    # initialization
    (n_samples,n_features) = np.shape(features)
    distance = np.zeros((n_samples,n_samples))
    weight = np.zeros(n_features)
    labels = list(map(int,labels))

    # compute distance
    for index_i in range(n_samples):
        for index_j in range(index_i+1,n_samples):
            D_value = features[index_i] - features[index_j]
            distance[index_i,index_j] = distanceNorm(norm,D_value)
    distance += distance.T

    # start iteration
    for iter_num in range(int(iter_ratio*n_samples)):
        # random extract a sample
        index_i = randrange(0,n_samples,1)
        self_features = features[index_i]

        # initialization
        nearHit = []
        nearMiss = dict()
        n_labels = list(set(labels))
        termination = np.zeros(len(n_labels))
        for label in n_labels:
            nearMiss[label] = []
        distance_sort = []


        # search for nearHit and nearMiss
        distance[index_i,index_i] = np.max(distance[index_i])		# filter self-distance 
        for index in range(n_samples):
            distance_sort.append([distance[index_i,index],index,labels[index]])

        distance_sort.sort(key = lambda x:x[0])

        for index in range(n_samples):
            # search nearHit
            if distance_sort[index][2] == labels[index_i]:
                if len(nearHit) < k:
                    nearHit.append(features[distance_sort[index][1]])
                else:
                    termination[distance_sort[index][2]] = 1
            # search nearMiss
            elif distance_sort[index][2] != labels[index_i]:
                if len(nearMiss[distance_sort[index][2]]) < k:
                    nearMiss[distance_sort[index][2]].append(features[distance_sort[index][1]])
                else:
                    termination[distance_sort[index][2]] = 1
            if len(list(termination)) == 0:
                break;

        # update weight
        nearHit_term = np.zeros(n_features)
        for x in nearHit:
            nearHit += np.abs(np.power(self_features - x,2))
        nearMiss_term = np.zeros((len(list(set(labels))),n_features))
        for index,label in enumerate(nearMiss.keys()):
            for x in nearMiss[label]:
                nearMiss_term[index] += np.abs(np.power(self_features - x,2))
            weight += nearMiss_term[index]/(k*len(nearMiss.keys()))
        weight -= nearHit_term/k

    return weight/(iter_ratio*n_samples)

def ReliefF(x_train, x_test, y_train, y_test, iter_ratio=10, k=3, norm=2):
    """
    'Flat' feature selection method Relief
    """
    weight = fit(x_train, y_train, iter_ratio, k, norm)
    x_train_permute = np.random.permutation(x_train)
    weight_permute = fit(x_train_permute, y_train, iter_ratio, k, norm)
        
    removed_idx = []
    for i in range(len(weight)):
        if weight[i]<=weight_permute[i]:
            removed_idx.append(i)

    x_train_rf = np.delete(x_train,removed_idx,1)
    x_test_rf = np.delete(x_test,removed_idx,1)
    
    # Modeling
    gm = modeling(x_train_rf,x_test_rf,y_train,y_test)
    return x_train_rf.shape[1],gm

## GTD

In [ ]:
def GTD(x_train, x_test, y_train, y_test,features,allPath,IVdict):
    """
    Greedy based top-down method
    """
    features_after = []
    for path in allPath:
        maxIV = 0
        feature_select = None
        for feature in path:
            if IVdict[feature] > maxIV:
                maxIV = IVdict[feature]
                feature_select = feature
        features_after.append(feature)
    
    features_removed = []
    for i,feature in enumerate(features):
        if feature not in features_after:
            features_removed.append(i)
            
    x_train_gtd = np.delete(x_train,features_removed,1)
    x_test_gtd = np.delete(x_test,features_removed,1)
    
    # Modeling
    gm = modeling(x_train_gtd,x_test_gtd,y_train,y_test)
    return x_train_gtd.shape[1],gm

## SHSEL

In [ ]:
def shselFS(x_train, y_train, F, H, parDict, IVdict, t):
    """
    Hierarchical Feature Selection Algorithm SHSEL
    Inputs: 
        x_train: training data
        y_train: training labels
        F: names of all features
        H: all longest paths in the feature hierarchy
        parDict: dictionary of {feature: [parents]}
        IVdict: dictionary of {feature:IG}
        t: similarity threshold
    Outputs:
        features_after: selected features
    """
    sim_func = lambda f1,f2: 1 - np.abs(IVdict[f1] - IVdict[f2])
    #Algoritm 1: initSHSEL
    for f in parDict:
        if f in F:
            for d in parDict[f]:
                if d in F:
                    sim = sim_func(f, d)
                    if sim >= t:
                        F[F.index(f)] = ''
                        break
     
    #Algorithm 2: pruneSHSEL
    features_after = set()
    for path in H:
        nodes = []
        IVs = []
        for node in path:
            if node in F:
                nodes.append(node)
                IVs.append(IVdict[node])
        avg = np.mean(IVs)
        for i,name in enumerate(nodes):
            if IVs[i] >= avg:
                features_after.add(node)
    features_after = list(features_after)
    return features_after

def SHSEL(x_train, x_test, y_train, y_test, features, allPath, parDict, IVdict, t=0.999):
    features_after = shselFS(x_train, y_train, features, allPath, parDict, IVdict, t)
    features_removed = []
    for i,feature in enumerate(features):
        if feature not in features_after:
            features_removed.append(i)
            
    x_train_shsel = np.delete(x_train,features_removed,1)
    x_test_shsel = np.delete(x_test,features_removed,1)
    
    # Modeling
    gms = modeling(x_train_shsel,x_test_shsel,y_train,y_test)
    return x_train_shsel.shape[1], gms

## ILP Method

In [ ]:
def preFiltering(x_train, x_test, y_train, y_test, features, allPath, IVdict):
    """
    Pre-filtering features by removing those that has a lower relevance score than its 'dummy twin'
    Inputs: 
        x_train: training data
        x_test: testing data
        y_train: training labels
        y_test: testing labels
        features: list of all features
        allPath: list of all paths
        IVdict: dictionary of {feature:IG}
    Outputs:
        x_train: training data
        x_test: testing data
        features: list of all features
        allPath: list of all paths
        ancDict: Dictionary of {feature: [ancestors]}, used in ILP method
        IVdict: dictionary of {feature:IG}
    """
    # Define the relevance function
    rel_func = lambda x,y: CalcIV(x,y) 
    
    removed_idx = []
    removed_features = []
    iv_permutes = []
    for i in range(x_train.shape[1]):
        iv = rel_func(x_train[:,i],y_train)
        # Randomly permute each feature 10 times
        for _ in range(10):
            # Each time calculate the IG of the permuted 'twin'
            x_train_permute = np.random.permutation(x_train)
            iv_permutes.append(rel_func(x_train_permute[:,i],y_train))
        # Remove unqualified features
        if iv <= np.median(iv_permutes):
            removed_idx.append(i)
            removed_features.append(features[i])
            
    for idx in removed_idx:
        features[idx] = ''
    while '' in features:
        features.remove('')
    x_train = np.delete(x_train,removed_idx,1)
    x_test = np.delete(x_test,removed_idx,1)

    for i,path in enumerate(allPath):
        for j,feature in enumerate(path):
            if feature not in features:
                allPath[i][j] = ''
        while '' in path:
            path.remove('')
        if len(path)<=1:
            allPath[i] = ''
    while '' in allPath:
        allPath.remove('')
    
    ancDict = dict()
    for f in features:
        ancDict[f] = set()
    for path in allPath:
        for i,node in enumerate(path):
            ancDict[node].update(path[:i])
    for l in ancDict:
        ancDict[l] = list(ancDict[l])
    
    for f in removed_features:
        IVdict.pop(f)
    return x_train,x_test,features,allPath,ancDict,IVdict

def findRel(x_train, y_train, features, allPath, ancDict, IVdict):
    """
    Calculate all relevance used in ILP method, i.e. labels' relevance and pairwise relevance
    Inputs:
        x_train: training data
        y_train: training labels
        features: list of all features
        allPath: list of all paths
        ancDict: Dictionary of {feature: [ancestors]}, used in ILP method
        IVdict: dictionary of {feature:IG}
    Outputs:
        features_rel: labels' relevance of all features
        features_depth: depths of all features
        pair_names: names of all child-ancestor pairs
        pair_rel: all pairwise relevance between features and ancestors
    """
    # Pairwise relevance calculation function
    sim_func = lambda f1,f2: CalcIV(x_train[:,features.index(f1)], x_train[:,features.index(f2)])
    
    # Calculate labels' relevance
    features_rel = []
    for feature in features:
        features_rel.append(IVdict[feature])
    
    # Calculate depths
    features_depth = []
    depth = dict()
    for f in features:
        depth[f] = []
    for path in allPath:
        for i, f in enumerate(path):
            depth[f].append(i)
    for f in features:
        if depth[f]==[]:
            features_depth.append(0)
        else:
            features_depth.append(np.mean(depth[f]))
    
    # Calculate pairwise relevance
    pair_names = []
    pair_rel = []
    for l in ancDict:
        for d in ancDict[l]:
            if l in features and d in features:
                pair_names.append((l,d))
                pair_rel.append(sim_func(l,d))
    return features_rel, features_depth, pair_names, pair_rel

def ilpFS(features, features_rel, features_depth, pair_names, pair_rel, lam, k):
    """
    Solve the ILP problem with Pulp
    Inputs:
        features: list of all features
        features_rel: list of relevance of each feature and its label
        pair_names: list of the names of feature-feature pairs: like [('GO:00001', 'GO:00002'), ('GO:00003', 'GO:00004')]
        pair_rel: list of relevance values of feature-feature pairs
        lam: the lambda value
        k: maximum number of features selected
    Output:
        features_removed: indexes of features that are removed
    """
    # Define an optimization problem
    prob = pulp.LpProblem('LP1' , pulp.LpMaximize)
    
    # unknown variables w's and z's
    ws = [pulp.LpVariable('W%d'%i , lowBound = 0 , upBound=1, cat = pulp.LpInteger) for i in range(0 , len(features_rel))]
    zs = [pulp.LpVariable('Z%d'%i , lowBound = 0 , upBound=1, cat = pulp.LpInteger) for i in range(0 , len(pair_rel))]
    
    # Objective Function
    objective = sum([features_rel[i]*ws[i]/(features_depth[i] + 1e-3) for i in range(len(features_rel))])\
                - lam * sum([ pair_rel[i]*zs[i] for i in range(len(pair_rel)) ])
    prob += objective
    
    # Constraints
    for i,pair in enumerate(pair_names):
        idx1 = features.index(pair[0])
        idx2 = features.index(pair[1])
        prob += (ws[idx1] + ws[idx2] - zs[i]) <= 1
        prob += (-ws[idx1] - ws[idx2] + 2*zs[i]) <= 0
    
    # Restrict the maximum number of features selected
    prob += sum(ws) <= k
    
    # Solve the problem
    prob.solve()
    
    features_removed = []
    for v in prob.variables()[:len(features)]:
        if v.varValue==0:
            features_removed.append(eval(v.name[1:]))
    return features_removed

def ILP(x_train, x_test, y_train, y_test, features, allPath, IVdict, k=10000, lam=0.01):
    # pre-filtering
    x_train, x_test, features, allPath, ancDict, IVdict = preFiltering(x_train, x_test, y_train, y_test, features, allPath, IVdict)
    
    # Child-ancestors ILP
    features_rel, features_depth, pair_names, pair_rel = findRel(x_train, y_train, features, allPath, ancDict, IVdict)
    features_removed = ilpFS(features, features_rel, features_depth, pair_names, pair_rel, lam, k)
    x_train = np.delete(x_train,features_removed,1)
    x_test = np.delete(x_test,features_removed,1)

    # Modeling
    gm = modeling(x_train, x_test, y_train, y_test)
    return x_train.shape[1], gm

# Algorithms Comparison

In [ ]:
def exp(datasets, freq=20):
    """
    For each dataset, randomly split it into train/test sets 20 times, and compute an average modeling score.
    """
    nofsGM = []
    rfN = []
    rfGM = []
    gtdN = []
    gtdGM = []
    shselN = []
    shselGM = []
    ilpN = []
    ilpGM = []
    ilpGM2 = []
    for dataset in datasets:
        print(dataset)
        x, y, features = dataInitialize(dataset)
        allPath, parDict = pathInitialize(dataset,features)
        nogm, rfgm, gtdgm, shgm, ilpgm, ilpgm2 = [], [], [], [], [], []
        rfn, gtdn, shn, ilpn = [], [], [], []
        for i in range(freq):
            print(i)
            x_train, x_test, y_train, y_test, IVdict = randomSplit(x,y,features)
            data = copy.deepcopy((x_train, x_test, y_train, y_test, features, allPath, IVdict))
            
            # No Feature selection
            gm = NOFS(x_train, x_test, y_train, y_test)
            nogm.append(gm)
            
            # ReliefF
            n,gm = ReliefF(x_train, x_test, y_train, y_test)
            rfn.append(n)
            rfgm.append(gm)

            # GTD
            n,gm = GTD(x_train, x_test, y_train, y_test, features, allPath, IVdict)
            gtdn.append(n)
            gtdgm.append(gm)
            x_train, x_test, y_train, y_test, features, allPath, IVdict = copy.deepcopy(data)
            
            # SHSEL
            nshsel, gm = SHSEL(x_train, x_test, y_train, y_test, features, allPath, parDict, IVdict)
            shn.append(nshsel)
            shgm.append(gm)
            x_train, x_test, y_train, y_test, features, allPath, IVdict = copy.deepcopy(data)

            # ILP
            n, gm = ILP(x_train, x_test, y_train, y_test, features, allPath, IVdict)
            ilpn.append(n)
            ilpgm.append(gm)
            x_train, x_test, y_train, y_test, features, allPath, IVdict = copy.deepcopy(data)
            
            # ILP-same number of features as SHSEL
            n, gm = ILP(x_train, x_test, y_train, y_test, features, allPath, IVdict, k=nshsel)
            ilpgm2.append(gm)
            x_train, x_test, y_train, y_test, features, allPath, IVdict = copy.deepcopy(data)
            
        nofsGM.append(getMean(nogm))
        rfN.append(getMean(rfn))
        rfGM.append(getMean(rfgm))
        gtdN.append(getMean(gtdn))
        gtdGM.append(getMean(gtdgm))
        shselN.append(getMean(shn))
        shselGM.append(getMean(shgm))
        ilpN.append(getMean(ilpn))
        ilpGM.append(getMean(ilpgm))
        ilpGM2.append(getMean(ilpgm2))
        
    return (nofsGM, rfN, rfGM, gtdN, gtdGM, shselN, shselGM, ilpN, ilpGM, ilpGM2)

In [ ]:
def run():
    pres = ['Worm/CE-']
    mids = ['BP','MF','CC','BPMF','BPCC','MFCC','BPMFCC']
    pro = '.txt'
    datasets = []
    names = []
    for pre in pres:
        for mid in mids:
            datasets.append(pre + mid + pro)
            names.append(pre + mid)
    nofsGM, rfN, rfGM, gtdN, gtdGM, shselN, shselGM, ilpN, ilpGM, ilpGM2 = exp(datasets)
    df = pd.DataFrame({'Datasets':names,'NOFS_GM':nofsGM, 'ReliefF_N':rfN, 'ReliefF_GM':rfGM, 'GTD_N':gtdN, 'GTD_GM':gtdGM, 'SHSEL_N':shselN, 'SHSEL_GM':shselGM, 'ILP_N':ilpN, 'ILP_GM':ilpGM, 'ILP_GM2':ilpGM2})
    return df

In [ ]:
df = run()
df
# df.to_csv('result.csv', index=False)